In [10]:
import pandas as pd
import pyodbc
import numpy as np
import joblib
import warnings
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
from pathlib import Path
warnings.filterwarnings("ignore")

# ---------------------------
# Load Data from SQL Server
# ---------------------------
def load_growth_data():
    df=pd.read_csv("C:/Users/vpotla/Desktop/Testing-SQL-Server/data.csv")
    # Create Date column
    df["Date"] = pd.to_datetime(df["yr"].astype(str) + "-" + df["mn"].astype(str) + "-01")
    df = df.sort_values("Date")
    
    # Aggregate (server+db usage % over time)
    group = df.groupby("Date")["per"].mean().reset_index()
    group = group.set_index("Date").asfreq("MS")  # monthly series
    group["per"] = group["per"].fillna(method="ffill")
    
    return group


# ---------------------------
# Model Selection (ARIMA)
# ---------------------------
def select_best_arima(series, p_range=(0, 8), d_range=(0, 3), q_range=(0, 8)):
    best_aic = float("inf")
    best_order = None
    best_model = None
    
    for p in range(*p_range):
        for d in range(*d_range):
            for q in range(*q_range):
                try:
                    model = ARIMA(series, order=(p, d, q)).fit()
                    if model.aic < best_aic:
                        best_aic = model.aic
                        best_order = (p, d, q)
                        best_model = model
                except:
                    continue
    
    return best_model, best_order, best_aic


# ---------------------------
# Save Only Model Params + Metadata
# ---------------------------
def save_model_metadata(model, order, aic, mse, mae, filename="best_model_meta.pkl"):
    metadata = {
        "order": order,
        "aic": aic,
        "mse": mse,
        "mae": mae,
        "params": model.params.to_dict()  # save only coefficients
    }
    joblib.dump(metadata, filename)
    print(f"Model metadata saved as {filename}")


# ---------------------------
# Main Execution
# ---------------------------
if __name__ == "__main__":
    # Load data
    df = load_growth_data()
    series = df["per"]

    # Split Train/Test (last 12 months as test)
    train_size = int(len(series) * 0.8)
    train, test = series.iloc[:train_size], series.iloc[train_size:]
    
    # Run ARIMA selection
    best_model, best_order, best_aic = select_best_arima(
        train, p_range=(0, 8), d_range=(0, 3), q_range=(0, 8)
    )
    print(f"Best ARIMA order={best_order}, AIC={best_aic}")
    
    # Forecast on test set
    forecast = best_model.forecast(steps=len(test))
    mse = mean_squared_error(test, forecast)
    mae = mean_absolute_error(test, forecast)
    print(f"Test MSE={mse}, MAE={mae}")
    
    # Save only metadata (not full training data)
    save_model_metadata(best_model, best_order, best_aic, mse, mae, "best_arima_meta2.pkl")


Best ARIMA order=(7, 1, 6), AIC=-56.75837258964576
Test MSE=0.008350014708253327, MAE=0.0700662328559645
Model metadata saved as best_arima_meta2.pkl


In [11]:
import pandas as pd
import numpy as np
import joblib
import warnings
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error

warnings.filterwarnings("ignore")

# ---------------------------
# Load CSV Data
# ---------------------------
def load_growth_data(file_path):
    df = pd.read_csv(file_path)
    # Create a Date column
    df["Date"] = pd.to_datetime(df["Year"].astype(str) + "-" + df["Month"].astype(str) + "-01")
    df = df.sort_values("Date")
    return df

# ---------------------------
# Select Best ARIMA per series
# ---------------------------
def select_best_arima(ts, p_range=(0,8), d_range=(0,3), q_range=(0,8)):
    best_aic = float("inf")
    best_order = None
    best_model = None
    
    for p in range(*p_range):
        for d in range(*d_range):
            for q in range(*q_range):
                try:
                    model = ARIMA(ts, order=(p,d,q)).fit()
                    if model.aic < best_aic:
                        best_aic = model.aic
                        best_order = (p,d,q)
                        best_model = model
                except:
                    continue
    return best_model, best_order, best_aic

# ---------------------------
# Forecast Growth% for a series
# ---------------------------
def forecast_growth(df, server, database, months_ahead=6):
    ts = df[(df["ServerName"]==server) & (df["DatabaseName"]==database)]
    ts = ts.set_index("Date")["Growth%"].asfreq("MS").fillna(method="ffill")
    
    if len(ts) < 12:  # Not enough data for ARIMA
        print(f"Skipping {server}-{database}: Not enough data")
        return None, None, None, None, None
    
    # Train/Test split
    train_size = int(len(ts) * 0.8)
    train, test = ts.iloc[:train_size], ts.iloc[train_size:]
    
    # Select best ARIMA
    model, order, aic = select_best_arima(train)
    
    # Forecast future months
    forecast = model.forecast(steps=months_ahead)
    
    # Evaluate on test set
    if len(test) > 0:
        pred_test = model.forecast(steps=len(test))
        mse = mean_squared_error(test, pred_test)
        mae = mean_absolute_error(test, pred_test)
    else:
        mse = mae = None
    
    return forecast, model, order, aic, mse, mae

# ---------------------------
# Save Metadata
# ---------------------------
def save_model_metadata(model, server, database, order, aic, mse, mae):
    metadata = {
        "server": server,
        "database": database,
        "order": order,
        "aic": aic,
        "mse": mse,
        "mae": mae,
        "params": model.params.to_dict()
    }
    filename = f"{server}_{database}_arima_meta.pkl"
    joblib.dump(metadata, filename)
    print(f"Saved metadata: {filename}")

# ---------------------------
# Main Execution
# ---------------------------
if __name__ == "__main__":
    file_path = "C:/Users/vpotla/Desktop/Testing-SQL-Server/data.csv"
    df = load_growth_data(file_path)
    
    months_ahead = 6  # Forecast horizon
    
    # Loop through all server/database combinations
    for server, database in df[["ServerName","DatabaseName"]].drop_duplicates().values:
        forecast, model, order, aic, mse, mae = forecast_growth(df, server, database, months_ahead)
        if forecast is not None:
            print(f"\nServer={server}, Database={database}")
            print(f"Best ARIMA order={order}, AIC={aic}")
            print(f"Forecast for next {months_ahead} months:\n{forecast}")
            print(f"MSE={mse}, MAE={mae}")
            
            # Save metadata only
            save_model_metadata(model, server, database, order, aic, mse, mae)



Server=Server1, Database=DB3
Best ARIMA order=(2, 1, 3), AIC=-128.18743449469758
Forecast for next 6 months:
2024-08-01    0.341238
2024-09-01    0.446361
2024-10-01    0.364392
2024-11-01    0.341477
2024-12-01    0.446349
2025-01-01    0.364166
Freq: MS, Name: predicted_mean, dtype: float64
MSE=0.006695556721624477, MAE=0.07007129537762759
Saved metadata: Server1_DB3_arima_meta.pkl

Server=Server1, Database=DB4
Best ARIMA order=(4, 0, 4), AIC=44.05952884249569
Forecast for next 6 months:
2024-09-01    0.042435
2024-10-01    0.390065
2024-11-01    0.788290
2024-12-01    0.238635
2025-01-01    0.249732
2025-02-01    0.715286
Freq: MS, Name: predicted_mean, dtype: float64
MSE=0.28769998574822336, MAE=0.41394449081547474
Saved metadata: Server1_DB4_arima_meta.pkl

Server=Server1, Database=DB2
Best ARIMA order=(4, 0, 3), AIC=-60.45004666177826
Forecast for next 6 months:
2024-09-01    0.286761
2024-10-01    0.344027
2024-11-01    0.320929
2024-12-01    0.295102
2025-01-01    0.263465
202